In [105]:
import ollama

EMBEDED_MODEL = "all-minilm:latest"
ollama.pull(EMBEDED_MODEL)
_ = [
    print(model["name"])
    for model in ollama.list()["models"]
    if model["name"].startswith("all-minilm")
]


def embed(prompt: str) -> list[float]:
    """Embed a text into a vector.
    """
    return ollama.embeddings(
        model=EMBEDED_MODEL,
        prompt=prompt,
    )["embedding"]


all-minilm:latest


# ทดลองทำ Vector embedding

In [106]:
text = "Hello world!"
embedded_text = embed(text)
vector_length = len(embedded_text)

print(f"Text: {text}")
print(f"Length vector: {vector_length}")
print(f"Embedded: {', '.join([str(e) for e in embedded_text[:10]])}, ...")

Text: Hello world!
Length vector: 384
Embedded: -0.10963227599859238, 0.13532641530036926, -0.0036588571965694427, 0.06237631291151047, -0.20462030172348022, -0.6458368301391602, 0.2243037223815918, -0.11236012727022171, -0.31714504957199097, 0.13018758594989777, ...


#  เปรียบเทียบระยะห่างระหว่าง Vector ด้วย Euclidean distance

In [107]:
import numpy as np
from scipy.spatial.distance import euclidean

texts = [
    "I'm going to have lunch",
    "I'm going to have dinner",
    "I want to go to Thailand this holiday",
]


for i in range(len(texts)):
    for j in range(i+1, len(texts)):
        text_1 = texts[i]
        text_2 = texts[j]
        print(f"""The distance between:
"{text_1}"
and
"{text_2}"
is {euclidean(embed(text_1), embed(text_2))}
----------------
""")


The distance between:
"I'm going to have lunch"
and
"I'm going to have dinner"
is 2.9729728622463747
----------------

The distance between:
"I'm going to have lunch"
and
"I want to go to Thailand this holiday"
is 6.982158763398947
----------------

The distance between:
"I'm going to have dinner"
and
"I want to go to Thailand this holiday"
is 7.226857975481565
----------------



In [108]:
import pandas as pd


movies_df = pd.read_csv("tmdb_5000_movies.csv")[["title", "overview"]]

In [109]:
movies_df.head(5)

,title,overview
0,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,Spectre,A cryptic message from Bond’s past sends him o...
3,The Dark Knight Rises,Following the death of District Attorney Harve...
4,John Carter,"John Carter is a war-weary, former military ca..."


In [110]:
# https://github.com/jina-ai/vectordb/
from docarray import BaseDoc
from docarray.typing import NdArray
from docarray import DocList
import numpy as np
import pandas as pd
from vectordb import InMemoryExactNNVectorDB, HNSWVectorDB
import shutil

class Movie(BaseDoc):
    title: str = ""
    overview: str = ""
    embedding: NdArray[vector_length]


try:
    shutil.rmtree("./workspace_path")
except FileNotFoundError:
    pass
# Specify your workspace path
db = HNSWVectorDB[Movie](workspace="./workspace_path")

# Index a list of documents with random embeddings
doc_list = []
for movie in movies_df.iloc:
    title = movie["title"]
    overview = movie["overview"]
    if pd.isna(overview):
        continue
    embedding = embed(overview)
    movie_doc = Movie(
        title=title,
        overview=overview,
        embedding=embedding
    )
    doc_list.append(movie_doc)

db.index(inputs=DocList[Movie](doc_list))

<DocList[Movie] (length=4800)>

In [89]:
query = "A spy goes on a mission"
query_doc = Movie(embedding=embed(query))
search_results = db.search(inputs=DocList[Movie]([query_doc]), limit=10)
for result in search_results[0].matches:
    print(f"Title: {result.title}")
    print(f"Overview: {result.overview}")
    print("-" * 16)

Title: 3 Days to Kill
Overview: A dangerous international spy is determined to give up his high stakes life to finally build a closer relationship with his estranged wife and daughter. But first, he must complete one last mission - even if it means juggling the two toughest assignments yet: hunting down the world's most ruthless terrorist and looking after his teenage daughter for the first time in ten years, while his wife is out of town.
----------------
Title: Act of Valor
Overview: When a covert mission to rescue a kidnapped CIA operative uncovers a chilling plot, an elite, highly trained U.S. SEAL team speeds to hotspots around the globe, racing against the clock to stop a deadly terrorist attack.
----------------
Title: Mission: Impossible
Overview: When Ethan Hunt, the leader of a crack espionage team whose perilous operation has gone awry with no explanation, discovers that a mole has penetrated the CIA, he's surprised to learn that he's the No. 1 suspect. To clear his name, Hu

In [97]:
def get_movie_contexts(prompt: str, limit: int = 5) -> list[Movie]:
    query_doc = Movie(embedding=embed(prompt))
    search_results = db.search(inputs=DocList[Movie]([query_doc]), limit=limit)
    return list(search_results[0].matches)


question = "Please recommend some movies about a spy going on a mission"
contexts = get_movie_context(question, limit=3)
for result in contexts:
    print(f"Title: {result.title}")
    print(f"Overview: {result.overview}")
    print("-" * 16)

Title: 3 Days to Kill
Overview: A dangerous international spy is determined to give up his high stakes life to finally build a closer relationship with his estranged wife and daughter. But first, he must complete one last mission - even if it means juggling the two toughest assignments yet: hunting down the world's most ruthless terrorist and looking after his teenage daughter for the first time in ten years, while his wife is out of town.
----------------
Title: Act of Valor
Overview: When a covert mission to rescue a kidnapped CIA operative uncovers a chilling plot, an elite, highly trained U.S. SEAL team speeds to hotspots around the globe, racing against the clock to stop a deadly terrorist attack.
----------------
Title: The Living Daylights
Overview: James Bond helps a Russian General escape into the west. He soon finds out that the KGB wants to kill him for helping the General. A little while later the General is kidnapped from the Secret Service leading 007 to be suspicious.
--

In [101]:
def chat(prompt: str, stream: bool = True) -> str:
    response = ollama.chat(
        model="qwen2:7b-instruct-q6_K",
        messages=[
            {"role": "user", "content": prompt},
        ],
        stream=stream,
    )
    
    if stream:
        results = ""
        for r in response:
            c = r["message"]["content"]
            print(c, end="")
            results += c
        return results
    else:
        return response["message"]["content"]

    
TEMPLATE = """
# Instruction
Answer user's question using given contexts

# Context:
{context}

# Question
{question}
"""

def build_prompt(question: str) -> str:
    movie_contexts = get_movie_contexts(question, limit=3)
    movie_contexts_prompt = ""
    for result in movie_contexts:
        movie_contexts_prompt += (f"Title: {result.title}\n")
        movie_contexts_prompt += (f"Overview: {result.overview}\n")
        movie_contexts_prompt += ("-" * 16)
    return TEMPLATE.format(context=movie_contexts_prompt, question=question)

prompt = build_prompt(question)
print(prompt)


# Instruction
Answer user's question using given contexts

# Context:
Title: 3 Days to Kill
Overview: A dangerous international spy is determined to give up his high stakes life to finally build a closer relationship with his estranged wife and daughter. But first, he must complete one last mission - even if it means juggling the two toughest assignments yet: hunting down the world's most ruthless terrorist and looking after his teenage daughter for the first time in ten years, while his wife is out of town.
----------------Title: Act of Valor
Overview: When a covert mission to rescue a kidnapped CIA operative uncovers a chilling plot, an elite, highly trained U.S. SEAL team speeds to hotspots around the globe, racing against the clock to stop a deadly terrorist attack.
----------------Title: The Living Daylights
Overview: James Bond helps a Russian General escape into the west. He soon finds out that the KGB wants to kill him for helping the General. A little while later the General 

In [103]:
from IPython.display import Markdown

Markdown(chat(prompt, stream=True))

Based on your interest in movies featuring spies going on missions, here are three recommendations:

1. **"3 Days to Kill"** - In this film, an international spy decides to retire from his high-stakes life to reconnect with his family but must complete one last mission.

2. **"Act of Valor"** - This movie centers around an elite U.S. SEAL team that embarks on a covert operation to rescue a CIA operative and thwart a deadly terrorist attack.

3. **"The Living Daylights"** - James Bond helps a Russian General escape from the Soviet Union, which leads him into a world of danger as he suspects foul play by both the KGB and his own agency.

These films all feature intense spy missions and thrilling action sequences that you might enjoy.

Based on your interest in movies featuring spies going on missions, here are three recommendations:

1. **"3 Days to Kill"** - In this film, an international spy decides to retire from his high-stakes life to reconnect with his family but must complete one last mission.

2. **"Act of Valor"** - This movie centers around an elite U.S. SEAL team that embarks on a covert operation to rescue a CIA operative and thwart a deadly terrorist attack.

3. **"The Living Daylights"** - James Bond helps a Russian General escape from the Soviet Union, which leads him into a world of danger as he suspects foul play by both the KGB and his own agency.

These films all feature intense spy missions and thrilling action sequences that you might enjoy.

In [104]:
from IPython.display import Markdown


prompt = build_prompt("I want to watch a romantic comedy movie. Do you have any recommendation?")
Markdown(chat(prompt, stream=True))

Yes, I can recommend a few romantic comedies for you:

1. **Title: 16 to Life**  
   Overview: This is a small-town romantic comedy about a teenager dealing with sexual inexperience on his birthday.

2. **Title: Midnight in Paris**  
   Overview: A romantic comedy involving a family traveling to the French capital, focusing on an engaged couple who grapples with the idea of whether a different life might be better for them.

3. **Title: About Last Night**  
   Overview: A contemporary romantic comedy that follows new love stories between two couples as they navigate their relationships from the initial stages to more serious commitments in real-life scenarios.

These movies offer various perspectives on romance and are likely to provide an enjoyable viewing experience based on your interest in romantic comedies.

Yes, I can recommend a few romantic comedies for you:

1. **Title: 16 to Life**  
   Overview: This is a small-town romantic comedy about a teenager dealing with sexual inexperience on his birthday.

2. **Title: Midnight in Paris**  
   Overview: A romantic comedy involving a family traveling to the French capital, focusing on an engaged couple who grapples with the idea of whether a different life might be better for them.

3. **Title: About Last Night**  
   Overview: A contemporary romantic comedy that follows new love stories between two couples as they navigate their relationships from the initial stages to more serious commitments in real-life scenarios.

These movies offer various perspectives on romance and are likely to provide an enjoyable viewing experience based on your interest in romantic comedies.